# VLM Cross-Modal Deep Hashing — Colab Launcher

Train a cross-modal hashing model (SigLIP2 → 1-bit binary codes) with real-time monitoring dashboard.

In [ ]:
# Cell 1: GPU Check + Google Drive Mount
import torch

assert torch.cuda.is_available(), "No GPU detected — switch to a GPU runtime."
gpu_name = torch.cuda.get_device_name(0)
vram = torch.cuda.get_device_properties(0).total_mem / 1024**3
print(f"GPU: {gpu_name} ({vram:.1f} GB)")

from google.colab import drive
drive.mount("/content/drive")

!mkdir -p /content/drive/MyDrive/vlm_quantization/checkpoints

In [ ]:
# Cell 2: Clone Repo + Install Dependencies + Load .env from Google Drive
!git clone https://github.com/hyunlord/vlm_quantization.git /content/vlm_quantization
%cd /content/vlm_quantization
!pip install -q -r requirements.txt
!pip install -q pyngrok

import os

env_path = "/content/drive/MyDrive/vlm_quantization/.env"
if os.path.exists(env_path):
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#") and "=" in line:
                key, val = line.split("=", 1)
                os.environ[key.strip()] = val.strip()
    print(f".env loaded from Google Drive")
else:
    print(f"No .env found at {env_path} — create one with NGROK_AUTH_TOKEN if needed")

In [ ]:
# Cell 3: Download COCO Captions 2014
!mkdir -p /content/data/coco/annotations
!wget -q http://images.cocodataset.org/zips/train2014.zip -O /tmp/train2014.zip
!wget -q http://images.cocodataset.org/zips/val2014.zip -O /tmp/val2014.zip
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2014.zip -O /tmp/ann.zip
!unzip -q /tmp/train2014.zip -d /content/data/coco/
!unzip -q /tmp/val2014.zip -d /content/data/coco/
!unzip -q /tmp/ann.zip -d /content/data/coco/
!rm /tmp/train2014.zip /tmp/val2014.zip /tmp/ann.zip
print("COCO Captions ready.")

In [ ]:
# Cell 4: Build Frontend (static export)
!apt-get -qq install -y nodejs npm > /dev/null 2>&1
!cd /content/vlm_quantization/monitor/frontend && npm install --silent && npm run build
print("Frontend built → monitor/frontend/out/")

In [ ]:
# Cell 5: Start Monitoring Server + ngrok Tunnel
import os
import threading
import time

import uvicorn
from pyngrok import ngrok

token = os.environ.get("NGROK_AUTH_TOKEN", "")
if token:
    ngrok.set_auth_token(token)

def run_server():
    uvicorn.run("monitor.server.app:app", host="0.0.0.0", port=8000, log_level="warning")

threading.Thread(target=run_server, daemon=True).start()
time.sleep(3)

tunnel = ngrok.connect(8000)
print(f"\n Dashboard: {tunnel.public_url}\n")

In [ ]:
# Cell 6: Train (checkpoints → Google Drive)
!cd /content/vlm_quantization && python train.py --config configs/colab.yaml

In [ ]:
# Cell 7: (Optional) Manual Checkpoint Backup
!cp -r /content/vlm_quantization/checkpoints/* /content/drive/MyDrive/vlm_quantization/checkpoints/ 2>/dev/null || echo "No local checkpoints to back up (already saving to Drive)."